In [1]:
import pandas as pd
import numpy as np
import itertools
import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier

from imblearn.metrics import geometric_mean_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import roc_curve
from sklearn.metrics import classification_report
from sklearn.metrics import auc
from sklearn.metrics import accuracy_score

pd.set_option('display.max_columns', 500)

### Define Functions

In [2]:
def plot_confusion_matrix(cm, classes, ax,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    Example taken from https://imbalanced-learn.readthedocs.io/en/stable/auto_examples/ensemble/plot_comparison_ensemble_classifier.html#sphx-glr-auto-examples-ensemble-plot-comparison-ensemble-classifier-py
    """
    print(cm)
    print('')

    ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.set_title(title)
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.sca(ax)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        ax.text(j, i, format(cm[i, j], fmt),
                horizontalalignment="center",
                color="white" if cm[i, j] > thresh else "black")

    ax.set_ylabel('True label')
    ax.set_xlabel('Predicted label')

### Load and prepare data

In [4]:
# Load training data
training_data_df = pd.read_csv("./Data/train.txt", sep = "\t")
#training_data_df.head(10)
#training_data_df.info()

In [5]:
# Convert categorical variable to strings
training_data_df['sku_department_id'] = training_data_df['sku_department_id'].astype(str)
training_data_df['ship_zone'] = training_data_df['ship_zone'].astype(str)
training_data_df['rec_ship_from_loc_cluster'] = training_data_df['rec_ship_from_loc_cluster'].astype(int).astype(str)
training_data_df['dest_cluster'] = training_data_df['dest_cluster'].astype(str)

# Show results
#training_data_df.head()
#training_data_df.info()

,orderYMD,sku_department_id,order_line_type,category,order_qty,loc_type_desc,loc_type_code,inventory_on_hand,days_in_transit,inventory_above_request,ship_zone,days_diff_planned,calc_days_diff_planned,sec_diff_eventTM_orderDT,day_of_week,week_of_year,diff_cut_grnd_event,diff_cut_air_event,exp_loc_qty_day,exp_loc_order_qty_day,rec_ship_from_loc_lat,rec_ship_from_loc_long,rec_ship_from_loc_cluster,dest_lat,dest_long,dest_cluster,recommended_distance_shipped,SHIP_WHEN_AVAIL_FLAG,PRE_ORDER_FLAG,upgrade_dummy
0,2018-11-16,6,NIB,BESTBUY,1,STORE,MS,461,3.0,460,5.0,11.0,10,2.833,Friday,46,-10236.833,-10236.833,3366099,1310,38.060691,-85.720904,21,29.740630,-95.829571,13,815.284384,N,N,False
1,2018-11-14,6,NIB,BESTBUY,1,STORE,MS,2,1.0,1,2.0,1.0,0,2.261,Wednesday,46,-18431.261,-18431.261,277,17,29.044149,-95.456131,13,30.178248,-95.500960,13,78.159317,N,N,False
2,2018-11-20,10,NIB,BESTBUY,1,STORE,SDS,2,1.0,1,4.0,6.0,5,5.200,Tuesday,47,7917.800,14217.800,13,7,30.472990,-81.643570,22,30.339837,-87.373783,6,342.225758,N,N,False
3,2018-11-20,5,NIB,BESTBUY,2,STORE,MS,37,1.0,35,2.0,7.0,6,5.278,Tuesday,47,-29771.278,-30671.278,128,14,33.727421,-84.757843,12,34.034515,-84.707349,12,21.363836,N,N,False
4,2018-11-17,10,NIB,RETAIL,1,DC,DC,1626,2.0,1625,4.0,2.0,1,557.562,Saturday,46,-31390.562,-31390.562,949769,1522,36.542556,-119.404051,9,41.194538,-111.906345,7,515.844914,N,N,True


#### Load test data

In [6]:
# Load test data
test_data_df = pd.read_csv("./Data/test.txt", sep = "\t")
#test_data_df.head(10)

In [7]:
# Convert categorical variable to strings
test_data_df['sku_department_id'] = test_data_df['sku_department_id'].astype(str)
test_data_df['ship_zone'] = test_data_df['ship_zone'].astype(str)
test_data_df['rec_ship_from_loc_cluster'] = test_data_df['rec_ship_from_loc_cluster'].astype(int).astype(str)
test_data_df['dest_cluster'] = test_data_df['dest_cluster'].astype(str)

# Show results
#test_data_df.head()

#### Load Validation Data

In [8]:
validation_data_df = pd.read_csv("./Data/validation.txt", sep = "\t")
#validation_data_df.head(10)

In [9]:
# Convert categorical variable to strings
validation_data_df['sku_department_id'] = validation_data_df['sku_department_id'].astype(str)
validation_data_df['ship_zone'] = validation_data_df['ship_zone'].astype(str)
validation_data_df['rec_ship_from_loc_cluster'] = validation_data_df['rec_ship_from_loc_cluster'].astype(int).astype(str)
validation_data_df['dest_cluster'] = validation_data_df['dest_cluster'].astype(str)

# Show results
#validation_data_df.head()

#### Fit Random Forest on just numerical features + SHIP_WHEN_AVAIL_FLAG

In [10]:
X_train = training_data_df[['order_qty', 
                            'inventory_on_hand',
                            'days_in_transit',
                            'inventory_above_request',
                            'calc_days_diff_planned',
                            'sec_diff_eventTM_orderDT',
                            'week_of_year',
                            'diff_cut_grnd_event',
                            'diff_cut_air_event',
                            'exp_loc_qty_day',
                            'exp_loc_order_qty_day',
                            'rec_ship_from_loc_lat',
                            'rec_ship_from_loc_long',
                            'dest_lat',
                            'dest_long',
                            'recommended_distance_shipped',
                            'loc_type_code',
                            'SHIP_WHEN_AVAIL_FLAG'
                           ]]

X_train = pd.get_dummies(X_train)

y_train = training_data_df['upgrade_dummy']

In [11]:
X_test = test_data_df[['order_qty', 
                       'inventory_on_hand',
                       'days_in_transit',
                       'inventory_above_request',
                       'calc_days_diff_planned',
                       'sec_diff_eventTM_orderDT',
                       'week_of_year',
                       'diff_cut_grnd_event',
                       'diff_cut_air_event',
                       'exp_loc_qty_day',
                       'exp_loc_order_qty_day',
                       'rec_ship_from_loc_lat',
                       'rec_ship_from_loc_long',
                       'dest_lat',
                       'dest_long',
                       'recommended_distance_shipped',
                       'loc_type_code',
                       'SHIP_WHEN_AVAIL_FLAG'
                      ]]

X_test = pd.get_dummies(X_test)

y_test = test_data_df['upgrade_dummy']

In [12]:
X_valid = validation_data_df[['order_qty', 
                              'inventory_on_hand',
                              'days_in_transit',
                              'inventory_above_request',
                              'calc_days_diff_planned',
                              'sec_diff_eventTM_orderDT',
                              'week_of_year',
                              'diff_cut_grnd_event',
                              'diff_cut_air_event',
                              'exp_loc_qty_day',
                              'exp_loc_order_qty_day',
                              'rec_ship_from_loc_lat',
                              'rec_ship_from_loc_long',
                              'dest_lat',
                              'dest_long',
                              'recommended_distance_shipped',
                              'loc_type_code',
                              'SHIP_WHEN_AVAIL_FLAG'
                              ]]

X_valid = pd.get_dummies(X_valid)

y_valid = validation_data_df['upgrade_dummy']

In [ ]:
#X_train = X_train[:500000]
#y_train = y_train[:500000]
#X_test = X_test.sample(frac=0.1, random_state=23)
#y_test = y_test.loc[y_test.index.isin(X_test.index)]

#### Fit random forest classifier

In [14]:
rf = RandomForestClassifier(n_estimators=10, 
                            random_state=0,
                            min_samples_split=15000,
                            min_samples_leaf = 250,
                            #max_depth=30,
                            max_features=6,
                            #oob_score=True,
                            n_jobs=-1)

rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=6, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=250, min_samples_split=15000,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [ ]:
max([estimator.tree_.max_depth for estimator in rf.estimators_])

In [ ]:
max([estimator.tree_.node_count for estimator in rf.estimators_])

In [ ]:
print('Score: ', rf.score(X_train, y_train))
#print('OOB Score: ', rf.oob_score_)

In [ ]:
y_pred_rf = rf.predict_proba(X_test)[:, 1]
y_pred_rf_no_probs = rf.predict(X_test)

In [ ]:
fpr_rf, tpr_rf, _ = roc_curve(y_test, y_pred_rf)

report2 = classification_report(y_test, y_pred_rf_no_probs)
print("Classification Report:")
print(report2)
print("Accuracy  Score : ", accuracy_score(y_test, y_pred_rf_no_probs))
print("Area under curve: ", auc(fpr_rf, tpr_rf))

In [ ]:
#Plot ROC Curve
plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_rf, tpr_rf, label='RF')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('Receiver operating characteristic')
#plt.legend(loc='best')
plt.show()

In [ ]:
print('Random Forest classifier performance:')
print('Balanced accuracy: {:.2f} - Geometric mean {:.2f}'
      .format(balanced_accuracy_score(y_test, y_pred_rf_no_probs),
              geometric_mean_score(y_test, y_pred_rf_no_probs)))
cm_rf = confusion_matrix(y_test, y_pred_rf_no_probs)
fig, ax = plt.subplots()
plot_confusion_matrix(cm_rf, classes=np.unique(y_train), ax=ax,
                      title='Random Forest')

In [ ]:
# get importances from RF
importances = rf.feature_importances_

# then sort them descending
indices = np.argsort(importances)[0:]

# get the features from the original data set
features = X_train.columns

# plot them with a horizontal bar chart
plt.figure(1)
plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), features[indices])
plt.xlabel('Relative Importance')
plt.show

In [ ]:
index_upgrade = np.where(y_test == True)
index_standard = np.where(y_test == False)

# Histogram predictions without error bars:
fig, ax = plt.subplots(1)
ax.hist(y_pred_rf[index_upgrade], histtype='step', label='upgrade')
# ax.hist(y_pred_rf[index_standard], histtype='step', label='standard')
ax.set_xlabel('Prediction (upgrade probability)')
ax.set_ylabel('Number of observations')
plt.legend()
plt.show()

In [ ]:
# Histogram predictions without error bars:
fig, ax = plt.subplots(1)
ax.hist(y_pred_rf[index_upgrade], histtype='step', label='upgrade')
ax.hist(y_pred_rf[index_standard], histtype='step', label='standard')
ax.set_xlabel('Prediction (upgrade probability)')
ax.set_ylabel('Number of observations')
plt.legend()
plt.show()

In [ ]:
# Compare prediction for out of sample cross-validation framework
y_valid_rf = rf.predict_proba(X_valid)[:, 1]
y_valid_rf_no_probs = rf.predict(X_valid)

fpr_valid_rf, tpr_valid_rf, _ = roc_curve(y_valid, y_valid_rf)

report3 = classification_report(y_valid, y_valid_rf_no_probs)
print("Classification Report:")
print(report3)
print("Accuracy  Score : ", accuracy_score(y_valid, y_valid_rf_no_probs))
print("Area under curve: ", auc(fpr_valid_rf, tpr_valid_rf))

In [ ]:
#Plot ROC Curve
plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_valid_rf, tpr_valid_rf, label='RF')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('Receiver operating characteristic')
#plt.legend(loc='best')
plt.show()

In [ ]:
print('Random Forest classifier performance:')
print('Balanced accuracy: {:.2f} - Geometric mean {:.2f}'
      .format(balanced_accuracy_score(y_valid, y_valid_rf_no_probs),
              geometric_mean_score(y_valid, y_valid_rf_no_probs)))
cm_rf = confusion_matrix(y_valid, y_valid_rf_no_probs)
fig, ax = plt.subplots()
plot_confusion_matrix(cm_rf, classes=np.unique(y_train), ax=ax,
                      title='Random Forest')

#### Fit balanced random forest classifier

In [ ]:
# brf = BalancedRandomForestClassifier(n_estimators=500, 
#                                      random_state=0,
#                                      min_samples_split=2500,
#                                      min_samples_leaf=250,
#                                      #max_depth=10,
#                                      max_features=5,
#                                      #oob_score=True,
#                                      n_jobs=-1)
brf = BalancedRandomForestClassifier(n_estimators=500, 
                                     random_state=0,
                                     min_samples_split=4000,
                                     min_samples_leaf=250,
                                     #max_depth=10,
                                     max_features=6,
                                     #oob_score=True,
                                     n_jobs=-1)

brf.fit(X_train, y_train)

In [ ]:
max([estimator.tree_.max_depth for estimator in brf.estimators_])

In [ ]:
max([estimator.tree_.node_count for estimator in brf.estimators_])

In [ ]:
print('Score: ', brf.score(X_train, y_train))
#print('OOB Score: ', brf.oob_score_)

In [ ]:
y_pred_brf = brf.predict_proba(X_test)[:, 1]
y_pred_brf_no_probs = brf.predict(X_test)

In [ ]:
fpr_brf, tpr_brf, _ = roc_curve(y_test, y_pred_brf)

report = classification_report(y_test, y_pred_brf_no_probs)
print("Classification Report:")
print(report)
print("Accuracy  Score : ", accuracy_score(y_test, y_pred_brf_no_probs))
print("Area under curve: ", auc(fpr_brf, tpr_brf))

In [ ]:
plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_rf, tpr_rf, label='RF')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('Receiver operating characteristic')
#plt.legend(loc='best')
plt.show()

In [ ]:
y_pred_brf_no_probs = brf.predict(X_test)

print('Balanced Random Forest classifier performance:')
print('Balanced accuracy: {:.2f} - Geometric mean {:.2f}'
      .format(balanced_accuracy_score(y_test, y_pred_brf_no_probs),
              geometric_mean_score(y_test, y_pred_brf_no_probs)))
cm_rf = confusion_matrix(y_test, y_pred_brf_no_probs)
fig, ax = plt.subplots()
plot_confusion_matrix(cm_rf, classes=np.unique(y_train), ax=ax,
                      title='Balanced Random Forest')

In [ ]:
# get importances from RF
importances = brf.feature_importances_

# then sort them descending
indices = np.argsort(importances)[0:]

# get the features from the original data set
features = X_train.columns

# plot them with a horizontal bar chart
plt.figure(1)
plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), features[indices])
plt.xlabel('Relative Importance')
plt.show

In [ ]:
index_upgrade = np.where(y_test == True)
index_standard = np.where(y_test == False)
y_pred_brf = brf.predict_proba(X_test)[:, 1]

# Histogram predictions without error bars:
fig, ax = plt.subplots(1)
ax.hist(y_pred_brf[index_upgrade], histtype='step', label='upgrade')
# ax.hist(y_pred_rf[index_standard], histtype='step', label='standard')
ax.set_xlabel('Prediction (upgrade probability)')
ax.set_ylabel('Number of observations')
plt.legend()
plt.show()

In [ ]:
# Histogram predictions without error bars:
fig, ax = plt.subplots(1)
ax.hist(y_pred_brf[index_upgrade], histtype='step', label='upgrade')
ax.hist(y_pred_brf[index_standard], histtype='step', label='standard')
ax.set_xlabel('Prediction (upgrade probability)')
ax.set_ylabel('Number of observations')
plt.legend()
plt.show()

In [ ]:
y_valid_brf = brf.predict_proba(X_valid)[:, 1]
y_valid_brf_no_probs = brf.predict(X_valid)

fpr_valid_brf, tpr_valid_brf, _ = roc_curve(y_valid, y_valid_brf)

report4 = classification_report(y_valid, y_valid_brf_no_probs)
print("Classification Report:")
print(report4)
print("Accuracy  Score : ", accuracy_score(y_valid, y_valid_brf_no_probs))
print("Area under curve: ", auc(fpr_valid_brf, tpr_valid_brf))

In [ ]:
print('Random Forest classifier performance:')
print('Balanced accuracy: {:.2f} - Geometric mean {:.2f}'
      .format(balanced_accuracy_score(y_valid, y_valid_brf_no_probs),
              geometric_mean_score(y_valid, y_valid_brf_no_probs)))
cm_rf = confusion_matrix(y_valid, y_valid_brf_no_probs)
fig, ax = plt.subplots()
plot_confusion_matrix(cm_rf, classes=np.unique(y_train), ax=ax,
                      title='Balanced Random Forest')

In [ ]:
plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_valid_brf, tpr_valid_brf, label='RF')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('Receiver operating characteristic')
#plt.legend(loc='best')
plt.show()

In [ ]:
y_valid_avg = 0.85*y_valid_brf + 0.15*y_valid_rf
y_valid_avg_no_probs = [val >= 0.5 for val in y_valid_avg]

In [ ]:
fpr_valid_avg, tpr_valid_avg, _ = roc_curve(y_valid, y_valid_avg)

report4 = classification_report(y_valid, y_valid_avg_no_probs)
print("Classification Report:")
print(report4)
print("Accuracy  Score : ", accuracy_score(y_valid, y_valid_avg_no_probs))
print("Area under curve: ", auc(fpr_valid_avg, tpr_valid_avg))

In [ ]:
plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_valid_brf, tpr_valid_brf, label='RF')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('Receiver operating characteristic')
#plt.legend(loc='best')
plt.show()

In [ ]:
print('Random Forest classifier performance:')
print('Balanced accuracy: {:.2f} - Geometric mean {:.2f}'
      .format(balanced_accuracy_score(y_valid, y_valid_avg_no_probs),
              geometric_mean_score(y_valid, y_valid_avg_no_probs)))
cm_rf = confusion_matrix(y_valid, y_valid_avg_no_probs)
fig, ax = plt.subplots()
plot_confusion_matrix(cm_rf, classes=np.unique(y_train), ax=ax,
                      title='Random Forest')

#### Perform Grid Search

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# Number of trees in random forest
n_estimators = [100, 250, 500]

# Number of features to consider at every split
max_features = np.arange(3,8)

# Maximum number of levels in tree
max_depth = [10, 25, 50, 100, 150, 200]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [50, 100, 200, 500]

# Minimum number of samples required at each leaf node
min_samples_leaf = [10, 50, 100, 200, 500]


# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf
              }

In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()

# Random search of parameters, using 2 fold cross validation, 
# search across 20 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, 
                               param_distributions = random_grid, 
                               n_iter = 20, 
                               cv = 2, 
                               verbose=2, 
                               random_state=42, 
                               n_jobs = -1
                              )

# Fit the random search model
rf_random.fit(X_train, y_train)

In [ ]:
# View the best parameters
rf_random.best_params_

#### Perform Cross Validation

In [ ]:
from sklearn.model_selection import GridSearchCV

# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}

# Create a based model
rf = RandomForestRegressor()

# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

#### Plot classification forest error bars

In [ ]:
import forestci as fci

In [ ]:
index_upgrade = np.where(y_test == True)
index_standard = np.where(y_test == False)
y_pred_rf = rf.predict_proba(X_test)[:, 1]

# Histogram predictions without error bars:
fig, ax = plt.subplots(1)
ax.hist(y_pred_rf[index_upgrade], histtype='step', label='upgrade')
ax.hist(y_pred_rf[index_standard], histtype='step', label='standard')
ax.set_xlabel('Prediction (upgrade probability)')
ax.set_ylabel('Number of observations')
plt.legend()

# Calcualte the variance
upgrade_variance_unbiased = fci.random_forest_error(rf, 
                                                    X_train, 
                                                    X_test,
                                                    memory_constrained = True,
                                                    memory_limit = 9000
                                                   )

# Plot forest prediction for emails and standard deviation for estimates
# Blue points are spam emails; Green points are non-spam emails
fig, ax = plt.subplots(1)
ax.scatter(y_pred_rf[index_upgrade],
           np.sqrt(upgrade_variance_unbiased[index_upgrade]),
           label='upgrade')

ax.scatter(y_pred_rf[index_standard],
           np.sqrt(upgrade_variance_unbiased[index_standard]),
           label='standard')

ax.set_xlabel('Prediction (upgrade probability)')
ax.set_ylabel('Standard deviation')
plt.legend()
plt.show()

In [ ]:
pred = np.array([tree.predict(X_test) for tree in rf]).T
print(max(np.mean(pred, 1)))
print(min(np.mean(pred, 1)))

In [ ]:
index_upgrade = np.where(y_test == True)
index_standard = np.where(y_test == False)

# Histogram predictions without error bars:
fig, ax = plt.subplots(1)
ax.hist(y_pred_brf[index_upgrade], histtype='step', label='upgrade')
ax.hist(y_pred_brf[index_standard], histtype='step', label='standard')
ax.set_xlabel('Prediction (upgrade probability)')
ax.set_ylabel('Number of observations')
plt.legend()

# Calcualte the variance
upgrade_variance_unbiased_brf = fci.random_forest_error(brf, 
                                                        X_train, 
                                                        X_test,
                                                        memory_constrained = True,
                                                        memory_limit = 9000
                                                       )

# Plot forest prediction for emails and standard deviation for estimates
# Blue points are spam emails; Green points are non-spam emails
fig, ax = plt.subplots(1)
ax.scatter(y_pred_brf[index_upgrade],
           np.sqrt(upgrade_variance_unbiased_brf[index_upgrade]),
           label='upgrade')

ax.scatter(y_pred_brf[index_standard],
           np.sqrt(upgrade_variance_unbiased_brf[index_standard]),
           label='standard')

ax.set_xlabel('Prediction (upgrade probability)')
ax.set_ylabel('Standard deviation')
plt.legend()
plt.show()

In [ ]:
pred = np.array([tree.predict(X_test) for tree in brf]).T
pred_mean = np.mean(pred, 1)
print("Max probability: ", max(pred_mean))
print("Min probability: ", min(pred_mean))